[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_var_calculation/EMQA_var_calculation.ipynb)

# EMQA_var_calculation
Value at Risk (VaR) with GARCH conditional volatility at 95% and 99% confidence.
**Output:** `var_calculation.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent returns
brent = fetch('BZ=F', start='2018-01-01')
log_ret = (np.log(brent / brent.shift(1)).dropna()) * 100

# GARCH conditional vol
try:
    from arch import arch_model
    am = arch_model(log_ret, vol='Garch', p=1, q=1, dist='normal', mean='Constant')
    res = am.fit(disp='off')
    cond_vol = res.conditional_volatility
    mu = res.params.get('mu', res.params.iloc[0])
except ImportError:
    omega, alpha, beta_ = 0.01, 0.08, 0.90
    mu = log_ret.mean()
    n = len(log_ret)
    cond_var = np.zeros(n)
    cond_var[0] = log_ret.var()
    ret_vals = log_ret.values
    for t in range(1, n):
        cond_var[t] = omega + alpha * ret_vals[t-1]**2 + beta_ * cond_var[t-1]
    cond_vol = pd.Series(np.sqrt(cond_var), index=log_ret.index)

from scipy.stats import norm
z_95 = norm.ppf(0.05)
z_99 = norm.ppf(0.01)

var_95 = mu + z_95 * cond_vol
var_99 = mu + z_99 * cond_vol

# Violations
viol_95 = log_ret < var_95
viol_99 = log_ret < var_99

print(f"VaR 95% violations: {viol_95.sum()} / {len(log_ret)} = {viol_95.mean()*100:.2f}% (expected 5%)")
print(f"VaR 99% violations: {viol_99.sum()} / {len(log_ret)} = {viol_99.mean()*100:.2f}% (expected 1%)")

In [ ]:
# Plot returns with VaR thresholds and violations
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(log_ret.index, log_ret.values, color=COLORS['gray'], linewidth=0.4, alpha=0.6, label='Returns')
ax.plot(var_95.index, var_95.values, color=COLORS['orange'], linewidth=1, label='VaR 95%')
ax.plot(var_99.index, var_99.values, color=COLORS['red'], linewidth=1, label='VaR 99%')

# Mark violations
viol_99_dates = log_ret.index[viol_99]
viol_99_vals = log_ret[viol_99]
ax.scatter(viol_99_dates, viol_99_vals, color=COLORS['red'], s=15, zorder=5, label='99% Exceedance')

viol_95_only = viol_95 & ~viol_99
viol_95_dates = log_ret.index[viol_95_only]
viol_95_vals = log_ret[viol_95_only]
ax.scatter(viol_95_dates, viol_95_vals, color=COLORS['orange'], s=10, zorder=4, label='95% Exceedance')

ax.set_xlabel('Date')
ax.set_ylabel('Returns (%)')
ax.set_title('Value at Risk - GARCH-based VaR on Brent Crude')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=5)

plt.tight_layout()
save_fig(fig, 'var_calculation.pdf')
plt.show()